In [1]:
import csv
import numpy as np 
import pandas as pd 
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [2]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,2,"Bell, Adele",female,62.00,0,0,PC 15008,14.86,D17243,C
99996,99996,0,2,"Brown, Herman",male,66.00,0,0,13273,11.15,NaN,S
99997,99997,0,3,"Childress, Charles",male,37.00,0,0,NaN,9.95,NaN,S
99998,99998,0,3,"Caughlin, Thomas",male,51.00,0,1,458654,30.92,NaN,S


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   PassengerId  100000 non-null  int64  
 1   Survived     100000 non-null  int64  
 2   Pclass       100000 non-null  int64  
 3   Name         100000 non-null  object 
 4   Sex          100000 non-null  object 
 5   Age          96708 non-null   float64
 6   SibSp        100000 non-null  int64  
 7   Parch        100000 non-null  int64  
 8   Ticket       95377 non-null   object 
 9   Fare         99866 non-null   float64
 10  Cabin        32134 non-null   object 
 11  Embarked     99750 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 9.2+ MB


In [4]:
train_df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,0.001956,-0.000893,-0.002654,0.001000,0.000411,-0.001036
Survived,0.001956,1.000000,-0.289723,0.103895,-0.007587,0.023352,0.187534
Pclass,-0.000893,-0.289723,1.000000,-0.259109,-0.051436,-0.041848,-0.417354
Age,-0.002654,0.103895,-0.259109,1.000000,-0.112459,-0.125535,0.109164
SibSp,0.001000,-0.007587,-0.051436,-0.112459,1.000000,0.213126,0.059358
Parch,0.000411,0.023352,-0.041848,-0.125535,0.213126,1.000000,0.061443
Fare,-0.001036,0.187534,-0.417354,0.109164,0.059358,0.061443,1.000000


In [5]:
train_df = train_df.drop(['PassengerId'], axis = 1)

In [6]:
# drop 'Name' in both train_df and test_df

train_df = train_df.drop(['Name'], axis = 1)

test_df = test_df.drop(['Name'], axis = 1)

In [7]:
# drop 'Ticket' in both train_df and test_df

train_df = train_df.drop(['Ticket'], axis = 1)

test_df = test_df.drop(['Ticket'], axis = 1)

#drop 'Cabin' in both train_df and test_df

train_df = train_df.drop(['Cabin'], axis = 1)

test_df = test_df.drop(['Cabin'], axis = 1)

In [8]:
data = [train_df, test_df]

for S in data:
    S['Sex'] = S['Sex'].replace({'male': '0'})
    S['Sex'] = S['Sex'].replace({'female': '1'})
    
#train_df

sns.barplot(x = 'Sex', y = 'Survived', palette = 'mako', data = train_df)

In [9]:
train_df.isnull().sum()

Survived        0
Pclass          0
Sex             0
Age          3292
SibSp           0
Parch           0
Fare          134
Cabin       67866
Embarked      250
dtype: int64

## most null value are Cabin > Age > Embarked > Fare in order

In [10]:
data = [train_df, test_df]

for SP in data:
    SP['#People'] = SP['SibSp'] + SP['Parch']
    #if alone
    SP.loc[SP['#People'] == 0, 'Status'] = 0
    #if have at least one person
    SP.loc[SP['#People'] > 0, 'Status'] = 1   
    SP['Status'] = SP['Status'].astype(int)
    
train_df['Status'].value_counts()

0    61853
1    38147
Name: Status, dtype: int64

In [11]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status
0,1,1,0,NaN,2,0,27.14,C12239,S,2,1
1,0,3,0,NaN,0,0,13.35,NaN,S,0,0
2,0,3,0,0.33,1,2,71.29,NaN,S,3,1
3,0,3,0,19.00,0,0,13.04,NaN,S,0,0
4,1,3,0,25.00,0,0,7.76,NaN,S,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,62.00,0,0,14.86,D17243,C,0,0
99996,0,2,0,66.00,0,0,11.15,NaN,S,0,0
99997,0,3,0,37.00,0,0,9.95,NaN,S,0,0
99998,0,3,0,51.00,0,1,30.92,NaN,S,1,1


### replace all null value

In [12]:
train_df['Fare'].describe()

count    99866.00000
mean        43.92933
std         69.58882
min          0.68000
25%         10.04000
50%         24.46000
75%         33.50000
max        744.66000
Name: Fare, dtype: float64

sns.distplot(train_df.Fare, color = 'Green', vertical = 'True') 

In [13]:
#replace null value in 'Fare' with it's mean
mean_fare = train_df['Fare'].mean()

data = [train_df, test_df]

for F in data:
    F['Fare'] = F['Fare'].fillna(mean_fare)
    F['Fare'] = F['Fare'].astype(int)
    
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status
0,1,1,0,NaN,2,0,27,C12239,S,2,1
1,0,3,0,NaN,0,0,13,NaN,S,0,0
2,0,3,0,0.33,1,2,71,NaN,S,3,1
3,0,3,0,19.00,0,0,13,NaN,S,0,0
4,1,3,0,25.00,0,0,7,NaN,S,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,62.00,0,0,14,D17243,C,0,0
99996,0,2,0,66.00,0,0,11,NaN,S,0,0
99997,0,3,0,37.00,0,0,9,NaN,S,0,0
99998,0,3,0,51.00,0,1,30,NaN,S,1,1


In [14]:
#'Fare' have an outlier so we will mainly focus in range of 0-105 
data = [train_df, test_df]

for F in data:
    F['Fare'] = F['Fare'].astype(int)
    F.loc[ F['Fare'] <= 15, 'Fare'] = 0 
    F.loc[(F['Fare'] > 15) & (F['Fare'] <= 30), 'Fare'] = 1 
    F.loc[(F['Fare'] > 30) & (F['Fare'] <= 45), 'Fare'] = 2 
    F.loc[(F['Fare'] > 45) & (F['Fare'] <= 60), 'Fare'] = 3 
    F.loc[(F['Fare'] > 60) & (F['Fare'] <= 75), 'Fare'] = 4 
    F.loc[(F['Fare'] > 75) & (F['Fare'] <= 90), 'Fare'] = 5 
    F.loc[(F['Fare'] > 90) & (F['Fare'] <= 105), 'Fare'] = 6 
    F.loc[ F['Fare'] > 105, 'Fare'] = 7

train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status
0,1,1,0,NaN,2,0,1,C12239,S,2,1
1,0,3,0,NaN,0,0,0,NaN,S,0,0
2,0,3,0,0.33,1,2,4,NaN,S,3,1
3,0,3,0,19.00,0,0,0,NaN,S,0,0
4,1,3,0,25.00,0,0,0,NaN,S,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,62.00,0,0,0,D17243,C,0,0
99996,0,2,0,66.00,0,0,0,NaN,S,0,0
99997,0,3,0,37.00,0,0,0,NaN,S,0,0
99998,0,3,0,51.00,0,1,1,NaN,S,1,1


In [15]:
train_df['Embarked'].describe()

count     99750
unique        3
top           S
freq      72139
Name: Embarked, dtype: object

sns.barplot(x = 'Embarked', y = 'Survived', palette = 'bright', data = train_df)

In [16]:
#set 'Embarked' group
data = [train_df, test_df]

for E in data:
    E['Embarked'] = E['Embarked'].replace({'C': '0'})
    E['Embarked'] = E['Embarked'].replace({'S': '1'})
    E['Embarked'] = E['Embarked'].replace({'Q': '2'})
    
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status
0,1,1,0,NaN,2,0,1,C12239,1,2,1
1,0,3,0,NaN,0,0,0,NaN,1,0,0
2,0,3,0,0.33,1,2,4,NaN,1,3,1
3,0,3,0,19.00,0,0,0,NaN,1,0,0
4,1,3,0,25.00,0,0,0,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,62.00,0,0,0,D17243,0,0,0
99996,0,2,0,66.00,0,0,0,NaN,1,0,0
99997,0,3,0,37.00,0,0,0,NaN,1,0,0
99998,0,3,0,51.00,0,1,1,NaN,1,1,1


In [17]:
#top value of 'Embarked' is 'S'
top_embarked = '1'
data = [train_df, test_df]

for E in data:
    E['Embarked'] = E['Embarked'].fillna(top_embarked)
    
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status
0,1,1,0,NaN,2,0,1,C12239,1,2,1
1,0,3,0,NaN,0,0,0,NaN,1,0,0
2,0,3,0,0.33,1,2,4,NaN,1,3,1
3,0,3,0,19.00,0,0,0,NaN,1,0,0
4,1,3,0,25.00,0,0,0,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,62.00,0,0,0,D17243,0,0,0
99996,0,2,0,66.00,0,0,0,NaN,1,0,0
99997,0,3,0,37.00,0,0,0,NaN,1,0,0
99998,0,3,0,51.00,0,1,1,NaN,1,1,1


In [18]:
train_df['Age'].describe()

count    96708.000000
mean        38.355472
std         18.313556
min          0.080000
25%         25.000000
50%         39.000000
75%         53.000000
max         87.000000
Name: Age, dtype: float64

In [19]:
#replace null value in 'Age' with it's mean
mean_age = round(train_df['Age'].mean()) 

data = [train_df, test_df]

for A in data:
    A['Age'] = A['Age'].fillna(mean_age)
    
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status
0,1,1,0,38.00,2,0,1,C12239,1,2,1
1,0,3,0,38.00,0,0,0,NaN,1,0,0
2,0,3,0,0.33,1,2,4,NaN,1,3,1
3,0,3,0,19.00,0,0,0,NaN,1,0,0
4,1,3,0,25.00,0,0,0,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,62.00,0,0,0,D17243,0,0,0
99996,0,2,0,66.00,0,0,0,NaN,1,0,0
99997,0,3,0,37.00,0,0,0,NaN,1,0,0
99998,0,3,0,51.00,0,1,1,NaN,1,1,1


In [20]:
#set 'Age' group max age = 87
data = [train_df, test_df]

for A in data:
    A['Age'] = A['Age'].astype(int)
    A.loc[ A['Age'] <= 20, 'Age'] = 0 #child
    A.loc[(A['Age'] > 20) & (A['Age'] <= 40), 'Age'] = 1 #teenager
    A.loc[(A['Age'] > 40) & (A['Age'] <= 60), 'Age'] = 2 #adult
    A.loc[(A['Age'] > 60) & (A['Age'] <= 80), 'Age'] = 3 #old person
    A.loc[ A['Age'] > 80, 'Age'] = 4 #very old

train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status
0,1,1,0,1,2,0,1,C12239,1,2,1
1,0,3,0,1,0,0,0,NaN,1,0,0
2,0,3,0,0,1,2,4,NaN,1,3,1
3,0,3,0,0,0,0,0,NaN,1,0,0
4,1,3,0,1,0,0,0,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,3,0,0,0,D17243,0,0,0
99996,0,2,0,3,0,0,0,NaN,1,0,0
99997,0,3,0,1,0,0,0,NaN,1,0,0
99998,0,3,0,2,0,1,1,NaN,1,1,1


In [21]:
train_df['Cabin'].describe()

count      32134
unique     26992
top       C19828
freq           6
Name: Cabin, dtype: object

train_df['Cabin Sign'] = train_df['Cabin'].apply(lambda x: str(x)[0])
train_df['Cabin Sign'].replace({'n':'A'}, inplace = True)

test_df['Cabin Sign'] = test_df['Cabin'].apply(lambda x: str(x)[0])
test_df['Cabin Sign'].replace({'n':'A'}, inplace = True)

train_test['Cabin Sign'] = train_test['Cabin'].apply(lambda x: str(x)[0])
train_test['Cabin Sign'].replace({'n':'A'}, inplace = True)

In [22]:
#set 'Cabin' group by it's max value (first letter of cabin zone)
data = [train_df, test_df]

for C in data:
   
    C['Cabin Sign'] = C['Cabin'].apply(lambda x: str(x)[0])
    C['Cabin Sign'].replace({'n':'A'}, inplace = True)

train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,#People,Status,Cabin Sign
0,1,1,0,1,2,0,1,C12239,1,2,1,C
1,0,3,0,1,0,0,0,NaN,1,0,0,A
2,0,3,0,0,1,2,4,NaN,1,3,1,A
3,0,3,0,0,0,0,0,NaN,1,0,0,A
4,1,3,0,1,0,0,0,NaN,1,0,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,1,3,0,0,0,D17243,0,0,0,D
99996,0,2,0,3,0,0,0,NaN,1,0,0,A
99997,0,3,0,1,0,0,0,NaN,1,0,0,A
99998,0,3,0,2,0,1,1,NaN,1,1,1,A


sns.barplot(x = 'Pclass', y = 'Survived', palette = 'Set3', data = train_df)

sns.barplot(x = 'Sex', y = 'Survived', palette = 'Set2', data = train_df)

sns.barplot(x = 'Age', y = 'Survived', palette = 'Set1', data = train_df)

sns.barplot(x = 'Fare', y = 'Survived', palette = 'pastel', data = train_df)

sns.barplot(x = 'Embarked', y = 'Survived', palette = 'dark', data = train_df)

## -- GENERATE MODEL --

In [23]:
X = train_df.drop(columns = ['Survived'])
y = train_df["Survived"]

print(X.shape, y.shape)

(100000, 11) (100000,)


In [24]:
from sklearn.preprocessing import LabelEncoder

cabin_le = LabelEncoder()

cabin_le.fit(X['Cabin Sign'])

X['Cabin Sign'] = cabin_le.transform(X['Cabin Sign'])

X = X.drop('Cabin', axis = 1)

X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#People,Status,Cabin Sign
0,1,0,1,2,0,1,1,2,1,2
1,3,0,1,0,0,0,1,0,0,0
2,3,0,0,1,2,4,1,3,1,0
3,3,0,0,0,0,0,1,0,0,0
4,3,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
99995,2,1,3,0,0,0,0,0,0,3
99996,2,0,3,0,0,0,1,0,0,0
99997,3,0,1,0,0,0,1,0,0,0
99998,3,0,2,0,1,1,1,1,1,0


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.20)

X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, random_state = 42, test_size = 0.50)

print(f'Training set: {X_train.shape}, {y_train.shape}')
print(f'Validation set: {X_valid.shape}, {y_valid.shape}')
print(f'Test set: {X_test.shape}, {y_test.shape}')

Training set: (80000, 10), (80000,)
Validation set: (10000, 10), (10000,)
Test set: (10000, 10), (10000,)


### DecisionTreeClassifier

In [26]:
# Create a decision tree model, named model, using DecisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", np.mean(y_pred == y_test))

Accuracy: 0.7635


### RandomForest

In [27]:
# Create a random forest model, named model, using RandomForest 
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100)

clf.fit(X_train,y_train)

from sklearn import metrics

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7635


### LogisticRegression

In [28]:
# Create a logistic regression model, named model, using LogisticRegression 
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'liblinear', random_state = 42)

model.fit(X_train,y_train)

print("Accuracy:", model.score(X_train,y_train))

Accuracy: 0.763975


### KNN

In [29]:
# Create a K-nearest neighbors model, named model, using KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 7) 

model.fit(X_train, y_train)  

print("Accuracy:", model.score(X_train,y_train))

Accuracy: 0.772875


### The Naive Bayes 

In [30]:
# Create the Naive Bayes model, named model, using GaussianNB 
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

model.fit(X_train, y_train)  
 
print("Accuracy:", model.score(X_train,y_train))

Accuracy: 0.7276875


### Perceptron

In [31]:
# Create the Perceptron model, named model, using Perceptron 
from sklearn.linear_model import Perceptron 

model = Perceptron(max_iter = 1000)

model.fit(X_train, y_train)

print("Accuracy:", model.score(X_train,y_train))

Accuracy: 0.6943375


 ### SGD

In [32]:
# Create the SGD model, named model, using SGDClassifier
from sklearn.linear_model import SGDClassifier

model = linear_model.SGDClassifier(max_iter = 7, tol = None)

model.fit(X_train, y_train)

print("Accuracy:", model.score(X_train,y_train))

Accuracy: 0.7582


### GradientBoosting

In [33]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators = 100)

clf.fit(X_train,y_train)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7635


### MLP

In [34]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state = 1, max_iter = 300)

model.fit(X_train,y_train)

print("Accuracy:", model.score(X_train,y_train)) 

Accuracy: 0.779


In [35]:
import joblib

joblib.dump(model,'first_model_MLP.joblib')

['first_model_MLP.joblib']

In [36]:
test_file  = test_df.drop("PassengerId", axis = 1).copy()

test_file['Cabin Sign'] = cabin_le.transform(test_file['Cabin Sign'])

test_file = test_file.drop('Cabin', axis = 1)

test_file

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#People,Status,Cabin Sign
0,3,0,0,0,0,4,1,0,0,0
1,3,1,2,0,0,0,1,0,0,0
2,1,1,0,0,0,2,0,0,0,1
3,2,0,1,0,0,0,1,0,0,0
4,1,1,0,0,2,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...
99995,3,1,1,0,0,0,2,0,0,0
99996,1,0,2,1,0,4,1,1,1,0
99997,3,0,2,0,0,0,1,0,0,0
99998,1,1,2,1,2,1,0,3,1,1


## ʕ•́ᴥ•̀ʔっwe choose MLP Classifier to be our model ♡

In [37]:
y_pred = model.predict(test_file) #prediction answer

accuracy_model = round(model.score(X_train,y_train) * 100, 2) 
print(round(accuracy_model,2,), "%")

77.9 %


In [38]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

prediction = cross_val_score(model, X_train, y_train, cv = 3)

In [39]:
np.mean (prediction)

0.7767249947739113

In [40]:
df = pd.read_csv("sample_submission.csv")

df["Survived"] =  y_pred

df.to_csv("sample_submission.csv", index = False)

second_test_df = pd.read_csv("test.csv")
second_test_df["Survived"] = y_pred

second_test_df

train_test = pd.concat([second_train_df , second_test_df]).reset_index(drop = True)

train_test

# 2nd round : Pseudolabelling

In [41]:
raw_test_df = pd.read_csv("test.csv")

In [42]:
pseudo_label = raw_test_df.copy()

pseudo_label['Survived'] = model.predict(test_file)

pseudo_label['Prob'] = model.predict_proba(test_file).max(axis = 1)

pseudo_label = pseudo_label[pseudo_label['Prob'] >= 0.80].drop('Prob', axis = 1)

pseudo_label

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,100000,3,"Holliday, Daniel",male,19.0,0,0,24745,63.01,NaN,S,0
2,100002,1,"Harris, Heather",female,19.0,0,0,25990,38.91,B15315,C,1
3,100003,2,"Larsen, Eric",male,25.0,0,0,314011,12.93,NaN,S,0
7,100007,3,"Payton, Brian",male,13.0,0,0,42031,9.41,NaN,S,0
9,100009,3,"Mickle, Johnny",male,20.0,0,0,39023,6.17,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99993,199993,3,"Leftwich, Don",male,45.0,0,0,42267,7.02,NaN,S,0
99994,199994,1,"Floer, Bridgette",female,20.0,0,0,PC 22839,188.81,B19044,C,1
99996,199996,1,"Brown, Howard",male,59.0,1,0,13004,68.31,NaN,S,0
99997,199997,3,"Lightfoot, Cameron",male,47.0,0,0,4383317,10.87,NaN,S,0


In [43]:
pseudo_label.to_csv('pseudo_labelled_data.csv', index = False)

In [44]:
#for working on Pseudolabelling

ps_label = pd.read_csv("pseudo_labelled_data.csv")

second_train_df = pd.read_csv("train.csv")

In [45]:
train_test = pd.concat([second_train_df , ps_label]).reset_index(drop = True)

train_test

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
153853,199993,0,3,"Leftwich, Don",male,45.00,0,0,42267,7.02,NaN,S
153854,199994,1,1,"Floer, Bridgette",female,20.00,0,0,PC 22839,188.81,B19044,C
153855,199996,0,1,"Brown, Howard",male,59.00,1,0,13004,68.31,NaN,S
153856,199997,0,3,"Lightfoot, Cameron",male,47.00,0,0,4383317,10.87,NaN,S


In [46]:
#train_df = train_df.drop(['PassengerId'], axis = 1)

train_test = train_test.drop(['PassengerId'], axis = 1)

In [47]:
#train_df = train_df.drop(['Name'], axis = 1)

#test_df = test_df.drop(['Name'], axis = 1)

train_test = train_test.drop(['Name'], axis = 1)

In [48]:
#train_df = train_df.drop(['Ticket'], axis = 1)

#test_df = test_df.drop(['Ticket'], axis = 1)

train_test = train_test.drop(['Ticket'], axis = 1)

In [49]:
data = [train_df, test_df, train_test]

for S in data:
    S['Sex'] = S['Sex'].replace({'male': '0'})
    S['Sex'] = S['Sex'].replace({'female': '1'})
    
#train_df

In [50]:
data = [train_df, test_df, train_test]

for SP in data:
    SP['#People'] = SP['SibSp'] + SP['Parch']
    #if alone
    SP.loc[SP['#People'] == 0, 'Status'] = 0
    #if have at least one person
    SP.loc[SP['#People'] > 0, 'Status'] = 1   
    SP['Status'] = SP['Status'].astype(int)
    
train_df['Status'].value_counts()

0    61853
1    38147
Name: Status, dtype: int64

In [51]:
#replace null value in 'Fare' with it's mean
mean_fare = train_df['Fare'].mean()

data = [train_df, test_df, train_test]

for F in data:
    F['Fare'] = F['Fare'].fillna(mean_fare)
    F['Fare'] = F['Fare'].astype(int)
    
#train_df

In [52]:
#'Fare' have an outlier so we will mainly focus in range of 0-105 
data = [train_df, test_df, train_test]

for F in data:
    F['Fare'] = F['Fare'].astype(int)
    F.loc[ F['Fare'] <= 15, 'Fare'] = 0 
    F.loc[(F['Fare'] > 15) & (F['Fare'] <= 30), 'Fare'] = 1 
    F.loc[(F['Fare'] > 30) & (F['Fare'] <= 45), 'Fare'] = 2 
    F.loc[(F['Fare'] > 45) & (F['Fare'] <= 60), 'Fare'] = 3 
    F.loc[(F['Fare'] > 60) & (F['Fare'] <= 75), 'Fare'] = 4 
    F.loc[(F['Fare'] > 75) & (F['Fare'] <= 90), 'Fare'] = 5 
    F.loc[(F['Fare'] > 90) & (F['Fare'] <= 105), 'Fare'] = 6 
    F.loc[ F['Fare'] > 105, 'Fare'] = 7

#train_df

In [53]:
#set 'Embarked' group
data = [train_df, test_df, train_test]

for E in data:
    E['Embarked'] = E['Embarked'].replace({'C': '0'})
    E['Embarked'] = E['Embarked'].replace({'S': '1'})
    E['Embarked'] = E['Embarked'].replace({'Q': '2'})
    
#train_df

In [54]:
#top value of 'Embarked' is 'S'
top_embarked = '1'
data = [train_df, test_df, train_test]

for E in data:
    E['Embarked'] = E['Embarked'].fillna(top_embarked)
    
#train_df

In [55]:
#replace null value in 'Age' with it's mean
mean_age = round(train_df['Age'].mean()) 

data = [train_df, test_df, train_test]

for A in data:
    A['Age'] = A['Age'].fillna(mean_age)
    
#train_df

In [56]:
#set 'Age' group max age = 87
data = [train_df, test_df, train_test]

for A in data:
    A['Age'] = A['Age'].astype(int)
    A.loc[ A['Age'] <= 20, 'Age'] = 0 #child
    A.loc[(A['Age'] > 20) & (A['Age'] <= 40), 'Age'] = 1 #teenager
    A.loc[(A['Age'] > 40) & (A['Age'] <= 60), 'Age'] = 2 #adult
    A.loc[(A['Age'] > 60) & (A['Age'] <= 80), 'Age'] = 3 #old person
    A.loc[ A['Age'] > 80, 'Age'] = 4 #very old

#train_df

In [57]:
#set 'Cabin' group by it's max value (first letter of cabin zone)
data = [train_df, test_df, train_test]

for C in data:
   
    C['Cabin Sign'] = C['Cabin'].apply(lambda x: str(x)[0])
    C['Cabin Sign'].replace({'n':'A'}, inplace = True)

#train_df

## -- GENERATE MODEL --

In [66]:
X = train_test.drop(columns = ['Survived'])
y = train_test["Survived"]

print(X.shape, y.shape)

(153858, 11) (153858,)


In [67]:
cabin_le = LabelEncoder()

cabin_le.fit(X['Cabin Sign'])

X['Cabin Sign'] = cabin_le.transform(X['Cabin Sign'])

X = X.drop('Cabin', axis = 1)

X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#People,Status,Cabin Sign
0,1,0,0,2,0,1,1,2,1,2
1,3,0,0,0,0,0,1,0,0,0
2,3,0,0,1,2,4,1,3,1,0
3,3,0,0,0,0,0,1,0,0,0
4,3,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
153853,3,0,2,0,0,0,1,0,0,0
153854,1,1,0,0,0,7,0,0,0,1
153855,1,0,2,1,0,4,1,1,1,0
153856,3,0,2,0,0,0,1,0,0,0


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.20)

X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, random_state = 42, test_size = 0.50)

print(f'Training set: {X_train.shape}, {y_train.shape}')
print(f'Validation set: {X_valid.shape}, {y_valid.shape}')
print(f'Test set: {X_test.shape}, {y_test.shape}')

Training set: (123086, 10), (123086,)
Validation set: (15386, 10), (15386,)
Test set: (15386, 10), (15386,)


## work on MLP classifier (our best model on the first time)

In [69]:
model = MLPClassifier(random_state = 1, max_iter = 300)

model.fit(X_train,y_train)

print("Accuracy:", model.score(X_train,y_train)) 

Accuracy: 0.8556781437368994


In [70]:
joblib.dump(model,'second_model_MLP.joblib')

['second_model_MLP.joblib']

In [71]:
test_file  = test_df.drop("PassengerId", axis = 1).copy()

test_file['Cabin Sign'] = cabin_le.transform(test_file['Cabin Sign'])

test_file = test_file.drop('Cabin', axis = 1)

test_file

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#People,Status,Cabin Sign
0,3,0,0,0,0,0,1,0,0,0
1,3,1,0,0,0,0,1,0,0,0
2,1,1,0,0,0,0,0,0,0,1
3,2,0,0,0,0,0,1,0,0,0
4,1,1,0,0,2,0,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...
99995,3,1,0,0,0,0,2,0,0,0
99996,1,0,0,1,0,0,1,1,1,0
99997,3,0,0,0,0,0,1,0,0,0
99998,1,1,0,1,2,0,0,3,1,1


In [72]:
y_pred = model.predict(test_file) #prediction answer

accuracy_model = round(model.score(X_train,y_train) * 100, 2) 
print(round(accuracy_model,2,), "%")

85.57 %


In [73]:
prediction = cross_val_score(model, X_train, y_train, cv = 3)

np.mean (prediction)

0.8537120549554117

In [74]:
df = pd.read_csv("sample_submission.csv")

df["Survived"] =  y_pred

df.to_csv("sample_submission.csv", index = False)

## --------------------------------------------------------------- END ---------------------------------------------------------------